# Jupyter Notebook for the generation of a suitable protein coding subseted genome

Run the required installs

In [2]:
!pip install pandas biopython gffpandas

In [3]:
from Bio import SeqIO
import gffpandas.gffpandas as gffpd
import gzip
import pandas as pd

In [51]:
reference_fasta = "../data/GRCh38.primary_assembly.genome.fa.gz"
reference_gtf = "../data/chr12_gencode_v44.annotation.gtf"

subset_coordinates = "chr12:6,487,526-6,616,396" 
chromosome, coords = subset_coordinates.split(":")

In [5]:
def read_gzipped_fasta(path: str) -> dict[str: dict[str: str]]:
    with gzip.open(path, "rt") as handle:
        fasta = SeqIO.to_dict(SeqIO.parse(handle, "fasta"))
    return fasta

In [7]:
fasta_record = read_gzipped_fasta(reference_fasta)


In [18]:
def subset_fasta(fasta_record: SeqIO.SeqRecord, subset_coordinates: str) -> SeqIO.SeqRecord:
    chromosome, coords = subset_coordinates.split(":")
    start, end = coords.split("-")
    start = int(start.replace(",", ""))
    end = int(end.replace(",", ""))
    return fasta_record[chromosome][start:end]

In [22]:
def write_fasta(fasta_record: SeqIO.SeqRecord, output_path: str) -> None:
    with open(output_path, "w") as handle:
        SeqIO.write(fasta_record, handle, "fasta")

In [23]:
subsetted_fasta = subset_fasta(fasta_record, subset_coordinates)
write_fasta(subsetted_fasta, "../data/chr12.genome.fasta")

On second run through I decided to try the full chr22 as it is smallest

In [53]:
write_fasta(fasta_record["chr22"], "../data/full_chr22.genome.fasta")

# Update GTF
At this point the gtf has already been decompressed and subsetted to the chromosome of relevance

In [24]:
def read_gtf(path: str) -> pd.DataFrame:
    gtf = gffpd.read_gff3(path)
    return gtf.df

Load and ensure that it just contains entries for your chromosome 

In [45]:
gtf_record = read_gtf(reference_gtf)
gtf_record.value_counts("seq_id")


seq_id
chr12    188637
Name: count, dtype: int64

In [46]:
def update_gtf_coordinates(gtf_record: pd.DataFrame, subset_coordinates: str) -> pd.DataFrame:
    chromosome, coords = subset_coordinates.split(":")
    start, end = coords.split("-")
    start = int(start.replace(",", ""))
    end = int(end.replace(",", ""))
    gtf_record["start"] = gtf_record["start"] - start
    gtf_record["end"] = gtf_record["end"] - start
    gtf_record = gtf_record[(gtf_record["start"] >= 0) & (gtf_record["end"] <= end - start)]
    gtf_record["seq_id"] = chromosome
    return gtf_record

In [47]:
updated_gtf = update_gtf_coordinates(gtf_record, subset_coordinates)

/tmp/ipykernel_8951/49183473.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtf_record["seq_id"] = chromosome


In [49]:
# write gtf file
updated_gtf.to_csv("../data/chr12.gtf", sep="\t", index=False, header=False, quoting=3)

In [54]:
chr22_gtf = "../data/chr22_gencode_v44.annotation.gtf"
chr22_gtf_record = read_gtf(chr22_gtf)
chr22_start = chr22_gtf_record["start"].min()
chr22_end = chr22_gtf_record["end"].max()
chr22_updated_gtf = update_gtf_coordinates(chr22_gtf_record, f"chr22:{chr22_start}-{chr22_end}")
chr22_updated_gtf.to_csv("../data/chr22.gtf", sep="\t", index=False, header=False, quoting=3)

In [56]:
chr22_start, chr22_end, chr22_end - chr22_start

(10736171, 50801309, 40065138)